This is used as a "second core" to perform seemingly parallel experiments. In this case we do stuff related to experiment 94.

In [1]:
import numpy as np
from numpy import zeros, eye, array, diag, exp, ones, tanh, cosh, dstack, log, hstack, vstack, sinh
from numpy.linalg import solve, norm
from numpy.random import choice, default_rng, randint, randn
from scipy.stats import multivariate_normal as MVN
from scipy.special import logsumexp, ndtr
from scipy.stats import uniform as udist
import math
import time
from copy import deepcopy

import matplotlib.pyplot as plt
from matplotlib import rc
from ipywidgets.widgets import IntSlider
from ipywidgets.widgets import interact
from warnings import resetwarnings, catch_warnings, filterwarnings
import seaborn as sns
import pickle

from Manifolds.GKManifoldNew import GKManifold, data_generator, find_point_on_manifold, find_point_on_manifold_from_θ
from markov_snippets import MSAdaptive, SMCAdaptive
from tangential_hug_functions import TangentialHugSampler
from tangential_hug_functions import HugTangentialMultivariate
from RWM import RWM
from utils import ESS_univariate
from utils import send_notification_to_iphone

In [2]:
seed = 1234
m = 20     # number of latent variables
d = m + 4  # dimensionality of x-component

In [3]:
def generate_settings(
    d=d, 
    N=2000,
    B=20,
    δ=0.01,
    δmin=1e-3,
    δmax=100.0,
    εmin=1e-16,
    min_pm=1e-4,
    maxiter=500,
    verbose=False,
    εs_fixed=None,
    adaptiveε=True,
    adaptiveδ=False,
    z0_manual=None,
    pm_target=0.23,
    pd_target=0.4,
    dm_target=0.01,
    pm_switch=0.1,
    prior_seed=seed,
    low_memory=True,
    integrator='rwm',
    εprop_switch=0.01,
    ε0_manual=None,
    quantile_value=0.9,
    initialization='manual',
    switch_strategy='pm',
    kernel='uniform',
    prior_scale=7,
    thinning=10,
    burnin=100,
    use_autograd=False,
    stopping_criterion={'εmin', 'pm'},
    resampling_seed=1234,
    mh_kernel_seed=1234,
    resampling_scheme='multinomial',
    prop_hug=0.5,
    projection_method='qr',
    metropolised=False,
    δadaptation_method='ap',
    prop_esjd=0.95,
    adaptiveB=False,
    Bmin=5,
    Bmax=100,
    proxy_ap_metric='dm',
    adaptiveN=False,
    NBbudget=None,
    Badaptation_method=None,
    adaptive_prophug=False,
    min_prop_hug=0.1,
    max_prop_hug=0.9):
    # here d represents m+4
    # Generate the manifold
    θ0        = array([3.0, 1.0, 2.0, 0.5])       # True parameter value on U(0, 10) scale.
    ystar     = data_generator(θ0, m, seed=seed)  # Observed data
    q         = MVN(zeros(d), eye(d))             # Proposal distribution for THUG
    resetwarnings()
    manifold  = GKManifold(ystar, kernel_type=kernel, use_autograd=use_autograd)
    # grab arguments
    arguments = locals()
    return arguments

In [5]:
δmin   = 1e-3
min_pm = 1e-4
pm_switch = 0.1

### B=20, N=2500, PH=0.5

In [7]:
with open("experiment93/second_particles10k.pkl", "rb") as file:
    P1_10k = pickle.load(file)

In [8]:
B20N2k_HAN_05 = MSAdaptive(generate_settings(d=d, N=2500, B=20,
        δ=0.01, δmin=δmin, min_pm=(1/2500), verbose=True, εs_fixed=None, adaptiveε=True, adaptiveδ=False,
        z0_manual=P1_10k[:2500], pm_target=0.23, pm_switch=pm_switch, low_memory=True, integrator='hug_and_nhug',
        εprop_switch=0.01, ε0_manual=200.0, quantile_value=0.5, initialization='manual',
        switch_strategy='pm', kernel='uniform',use_autograd=False, stopping_criterion={'εmin', 'pm', 'maxiter'},
        resampling_scheme='multinomial', maxiter=1000, δadaptation_method=None, adaptiveB=False,
        Bmin=5, Bmax=100, proxy_ap_metric='pm', prop_esjd=0.95, Badaptation_method=None, prop_hug=0.5))
B20N2k_HAN_05_PARTICLES = B20N2k_HAN_05.sample()

Integrator: HUG + NHUG.
Prop Hug  :  0.5
Stopping criterion:  maxiter, εmin, pm.
Resampling: MULTINOMIAL.
Iteration:  1
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 54.7133582196436166
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9580000000000000
	Particle Diversity: 0.4448000000000000
	Moved Diversity: 0.4261184000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  2
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 32.7793215055888965
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9500000000000000
	Particle Diversity: 0.4256000000000000
	Moved Diversity: 0.4043200000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  3
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 25.2901330622615070
	Weights computed and normalized.
	SMC Weights computed and normalized.
	

	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9364000000000000
	Particle Diversity: 0.4608000000000000
	Moved Diversity: 0.4314931200000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  24
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 9.2445308460114930
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9448000000000000
	Particle Diversity: 0.4600000000000000
	Moved Diversity: 0.4346080000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  25
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 8.8350240185153481
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9416000000000000
	Particle Diversity: 0.4436000000000000
	Moved Diversity: 0.4176937600000000
	Step-size kept fixed at: 0.0100000000000000
I

	Trajectories constructed.
	Epsilon: 3.0193407338575762
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9244000000000000
	Particle Diversity: 0.4520000000000000
	Moved Diversity: 0.4178288000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  47
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 2.9271677434446861
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9212000000000000
	Particle Diversity: 0.4584000000000000
	Moved Diversity: 0.4222780800000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  48
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 2.8565929879791461
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9096000000000000
	Particle Diversity: 0.4648000000000000
	Moved Diversity: 0.4227820

	Trajectories constructed.
	Epsilon: 1.8618630637972668
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9192000000000000
	Particle Diversity: 0.4712000000000000
	Moved Diversity: 0.4331270400000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  70
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 1.8350794976641929
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9184000000000000
	Particle Diversity: 0.4692000000000000
	Moved Diversity: 0.4309132800000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  71
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 1.8013160112246092
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9116000000000000
	Particle Diversity: 0.4524000000000000
	Moved Diversity: 0.4124078

	Trajectories constructed.
	Epsilon: 1.2867003178035716
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9104000000000000
	Particle Diversity: 0.4544000000000000
	Moved Diversity: 0.4136857600000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  93
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 1.2675540552169380
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9008000000000000
	Particle Diversity: 0.4700000000000000
	Moved Diversity: 0.4233760000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  94
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 1.2491098359127928
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9196000000000000
	Particle Diversity: 0.4544000000000000
	Moved Diversity: 0.4178662

...

	Trajectories constructed.
	Epsilon: 0.1272340191174767
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.0428000000000000
	Particle Diversity: 0.1592000000000000
	Moved Diversity: 0.0068137600000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  576
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.1272061992893390
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.0408000000000000
	Particle Diversity: 0.2116000000000000
	Moved Diversity: 0.0086332800000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  577
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.1271848152431655
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.1036000000000000
	Particle Diversity: 0.1372000000000000
	Moved Diversity: 0.01421

	Trajectories constructed.
	Epsilon: 0.1265894500006383
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.0164000000000000
	Particle Diversity: 0.4224000000000000
	Moved Diversity: 0.0069273600000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  599
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.1265707994392648
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.0400000000000000
	Particle Diversity: 0.1168000000000000
	Moved Diversity: 0.0046720000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  600
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.1265570563590387
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.0184000000000000
	Particle Diversity: 0.3332000000000000
	Moved Diversity: 0.00613

	Trajectories constructed.
	Epsilon: 0.1261535757781906
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.0876000000000000
	Particle Diversity: 0.0704000000000000
	Moved Diversity: 0.0061670400000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  622
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.1261300863056079
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.0228000000000000
	Particle Diversity: 0.5552000000000000
	Moved Diversity: 0.0126585600000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  623
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.1261091808889452
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.0232000000000000
	Particle Diversity: 0.4496000000000000
	Moved Diversity: 0.01043

	Trajectories constructed.
	Epsilon: 0.1253991444876548
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.0888000000000000
	Particle Diversity: 0.4736000000000000
	Moved Diversity: 0.0420556800000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  645
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.1253437052680798
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.1836000000000000
	Particle Diversity: 0.2048000000000000
	Moved Diversity: 0.0376012800000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  646
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.1253013970889212
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.0760000000000000
	Particle Diversity: 0.3156000000000000
	Moved Diversity: 0.02398

	Trajectories constructed.
	Epsilon: 0.1245079102756910
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.0576000000000000
	Particle Diversity: 0.3988000000000000
	Moved Diversity: 0.0229708800000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  668
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.1244749866543062
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.0416000000000000
	Particle Diversity: 0.3460000000000000
	Moved Diversity: 0.0143936000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  669
	Quantile Value:  0.5
	Trajectories constructed.
	Epsilon: 0.1244409904668955
	Weights computed and normalized.
	SMC Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.0164000000000000
	Particle Diversity: 0.5896000000000000
	Moved Diversity: 0.00966

In [9]:
with open("experiment94/B20N2k_HAN_05_ess.pkl", "wb") as file:
    pickle.dump(B20N2k_HAN_05.ESS, file)
    
with open("experiment94/B20N2k_HAN_05_ess_smc.pkl", "wb") as file:
    pickle.dump(B20N2k_HAN_05.ESS_SMC, file)
    
with open("experiment94/B20N2k_HAN_05_epsilons.pkl", "wb") as file:
    pickle.dump(B20N2k_HAN_05.εs, file)
    
with open("experiment94/B20N2k_HAN_05_pm.pkl", "wb") as file:
    pickle.dump(B20N2k_HAN_05.PROP_MOVED, file)
    
with open("experiment94/B20N2k_HAN_05_pd.pkl", "wb") as file:
    pickle.dump(B20N2k_HAN_05.P_DIVERSITY, file)
    
with open("experiment94/B20N2k_HAN_05_dm.pkl", "wb") as file:
    pickle.dump(B20N2k_HAN_05.DIV_MOVED, file)
    
with open("experiment94/B20N2k_HAN_05_K_RESAMPLED.pkl", "wb") as file:
    pickle.dump(B20N2k_HAN_05.K_RESAMPLED, file)
    
with open("experiment94/B20N2k_HAN_05_N_RESAMPLED.pkl", "wb") as file:
    pickle.dump(B20N2k_HAN_05.N_RESAMPLED, file)

In [10]:
send_notification_to_iphone("Second core finished: B=20, N=2k, PH=0.5")